In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
from bayes_opt import BayesianOptimization

In [ ]:
# Global variables
global dataset
global model
global current_class
global trf_cont
global tau
global eps_thresh
global lambda_reg
global upper_bound_cont
global steps

In [1]:
# Compute the error between the prediction of a plain image and the prediction of a transformed image
# - model: the CNN trained on pure data
# - dataset: the dataset of images to use for the optimization
# - cls: the class of the images to use for the optimization
# - trf_cont: the transformation to apply
# - phi: the argument of the transformation
def compute_delta_cont(phi):
    steps_no = 0
    error_sum = 0

    # Generate a vector of "steps" integer from 0 to len(dataset['data'])
    rnd_indexes = np.random.randint(0, len(dataset['data']), steps)

    predictions = 0
    misclassification = 0

    for i in rnd_indexes:

        while(dataset['labels'][i] != cls and i < len(dataset['data'])):
            i += 1
        image = dataset['data'][i]
        image = np.array(tf.image.resize(image, (224, 224)).numpy().astype(int))

        steps_no += 1
        if trf_cont == 'rotate':
            transformed_image1 = ((tf.keras.layers.RandomRotation((0.9*phi, phi))(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomRotation((-0.9*phi, 0))(image)).numpy()).astype(int)

        elif trf_cont == 'shear':
            transformed_image1 = tf.keras.preprocessing.image.random_shear(image, phi, row_axis=0, col_axis=1, channel_axis=2)
            transformed_image2 = tf.keras.preprocessing.image.random_shear(image, -phi, row_axis=0, col_axis=1, channel_axis=2)

        elif trf_cont == 'horizontal_shift':
            transformed_image1 = ((tf.keras.layers.RandomTranslation((0.9*phi, phi), 0)(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomTranslation((-phi, -0.9*phi), 0)(image)).numpy()).astype(int)

        elif trf_cont == 'vertical_shift':
            transformed_image1 = ((tf.keras.layers.RandomTranslation(0, (0.9*phi, phi))(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomTranslation(0, (-phi, -0.9*phi))(image)).numpy()).astype(int)

        elif trf_cont == 'zoom':
            transformed_image1 = ((tf.keras.layers.RandomZoom((0.9*phi, phi))(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomZoom((-phi, -0.9*phi))(image)).numpy()).astype(int)

        # Optimization to find the optimal upper bound for brightness transformation
        elif trf_cont == 'brightness_top':    
            transformed_image1 = ((tf.keras.layers.RandomBrightness((0.9*phi, phi))(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomBrightness((0.9*phi, phi))(image)).numpy()).astype(int)

        # Optimization to find the optimal lower bound for brightness transformation
        elif trf_cont == 'brightness_down':   
            transformed_image1 = ((tf.keras.layers.RandomBrightness((-phi, -0.9*phi))(image)).numpy()).astype(int)
            transformed_image2 = ((tf.keras.layers.RandomBrightness((-phi, -0.9*phi))(image)).numpy()).astype(int)
        
        elif trf_cont == 'contrast': 
            transformed_image1 = ((tf.keras.layers.RandomContrast(phi,phi))(image)).numpy().astype(int)
            transformed_image2 = ((tf.keras.layers.RandomContrast(phi,phi))(image)).numpy().astype(int)

        else:
            print('Transformation not found.')
            transformed_image1 = transformed_image2 = image

        if cls == 'healthy':
            prediction1 = model.predict(np.expand_dims(transformed_image1, axis=0))[0]
            prediction2 = model.predict(np.expand_dims(transformed_image2, axis=0))[0]

            # Compute the statistics
            # Checks if the prediction is wrong
            if 0 != np.argmax(prediction1):
                  misclassification += 1
            if 0 != np.argmax(prediction2):
                  misclassification += 1

            prediction1 = prediction1[0]
            prediction2 = prediction2[0]

        else:
            prediction1 = model.predict(np.expand_dims(transformed_image1, axis=0))[0]
            prediction2 = model.predict(np.expand_dims(transformed_image2, axis=0))[0]

            # Compute the statistics
            # Checks if the prediction is wrong
            if 1 != np.argmax(prediction1):
                  misclassification += 1
            if 1 != np.argmax(prediction2):
                  misclassification += 1

            prediction1 = prediction1[1]
            prediction2 = prediction2[1]

        # Compute the error value
        error_sum += (abs(1 - prediction1) + abs(1 - prediction2))
        predictions += 2

        # Stop the search after a fied number of steps
        if steps_no == steps:
              break

    mispredictions_ratio = misclassification/predictions

    predictions = 0
    misclassification = 0

    f = open("predictions.txt", "a")
    f.write("Class " + cls + " - Transformation " + trf_cont + ", misprediction ratio " +
            str(mispredictions_ratio) + " with phi " + str(phi) + "\n")
    f.close()

    # Error between the prediction with the pure image and the prediction with the transformed image
    delta = (1 / (2 * steps_no)) * error_sum

    return delta

In [ ]:
# Define the objective function for our optimization problem (funtions with continuous parameters)
# - phi: the argument of the transformation (parameter to be optimized)
# - dataset: the dataset to use for the optimization
# - trf_cont: the transformation to apply
# - model: the CNN trained on pure data
# - eps_thresh: the threshold for the softmax output
# - lambda_reg: balancing coefficient
# - upper_bound: the upper bound for the transformation
def ObjectiveFunction_cont(phi):
    # Get and sum the errors between the plain image and the transformed image
    delta = compute_delta_cont(phi)
    return min((eps_thresh - delta), 0) + lambda_reg * phi / upper_bound_cont

In [ ]:
from tensorflow.python.ops.array_ops import upper_bound
# Optimization based on paper "Class-Adaptive_Data_Augmentation_for_Image_Classification"
def AugmentationOptimizer(model_CNN, data, steps_no,  eps_thresh_value, lambda_reg_value):

    # Golbal variables
    global dataset
    dataset = data
    global model
    model = model_CNN
    global steps
    steps = steps_no

    # Create a vector with data transformations:
    # - up to gaussian noise, the values are in the range [x, y]
    transformations = {'rotate': [0, 0.5], 'shear': [0, 45], 'horizontal_shift': [0, 0.5], 'vertical_shift': [0, 0.5],
                       'zoom': [0, 0.5], 'brightness_top': [0, 1], 'brightness_down': [-1, 0], 'contrast': [0, 1]}

    # For each class
    print('Optimizing...')
    global eps_thresh
    eps_thresh =  eps_thresh_value
    global lambda_reg
    lambda_reg = lambda_reg_value
    for current_class in ['unhealthy']:
        global cls
        cls = current_class
        
        for transformation in transformations.keys():
            print("Optimizing continue transformations...")
            # Assign the global varables to be seen by the objective function
            global trf_cont
            trf_cont = transformation
            global upper_bound_cont
            upper_bound_cont = transformations[trf_cont][1]

            print('Class ' + cls + ' - Transformation ' + trf_cont)
            print('Bounds: ' + str(transformations[trf_cont][0]) + ' - ' + str(transformations[trf_cont][1]))

            # Define space for the parameters
            pbounds = {'phi' : (transformations[trf_cont][0], transformations[trf_cont][1])}
            # Initialize BayesianOptimization object
            optimizer = BayesianOptimization(
                f=ObjectiveFunction_cont,
                pbounds=pbounds,
                verbose=2,
                allow_duplicate_points=True
            )

            # Define the type of optimization
            optimizer.maximize(init_points=5,n_iter=10)

            # Retrieve the results
            optimal_params = optimizer.max['params']
            optimal_objective_value = optimizer.max['target']

            print("Optimal phi for " + str(trf_cont) + " for " + cls + " class: " + str(optimal_params))
            print("Optimal objective function value for " + str(trf_cont) + ": " + str(optimal_objective_value))


            # Save the results into a file
            f = open("results.txt", "a")
            f.write("Class " + cls + " - Transformation " + trf_cont + ": " + str(optimal_params) + "\n")
            f.close()
       

In [ ]:
# From typing_extensions import dataclass_transform
print('Loading model...')
model_CNN = tf.keras.models.load_model("ConvNeXtLarge_lr_0.001_bs_64_val_0.9185779690742493")
data = np.load("train_dataset_no_duplicate.npz", allow_pickle=True)

In [ ]:
# Call the optimizer
steps_no = 20 # Number of images used for each iteration
eps_thresh_value = 0.1 # Values suggested by the creators of the method
lambda_reg_value = 0.01
AugmentationOptimizer(model_CNN, data, steps_no, eps_thresh_value, lambda_reg_value)